# <font color='orange'>Trabajo práctico N° 3</font>
## <font color='cornflowerblue'>Visión por computadora I</font>
### <font color='violet'>Alumno: Zenklusen, Kevin</font>

Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en 
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template
1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección

1. El siguiente código permite detectar la ubicación del logo una vez en la imagen. En el caso de múltiples logos, indica la mejor región en la que se encuentra el conjunto.
<br>
Para realizar la detección se se preprocesa las imágenes quitando el ruido y utilizando el algoritmo Canny. Luego se escala y rota levemente el template para buscar los matches.
<br>
Esta función pretende ser un método básico de detección, y la filtración de falsos positivos se realiza mediante score.
<br>
El score se calcula mediante una comparación entre el template consigo mismo. Se consideró utilizar los métodos de búscqueda nomralizados (como TM_CCOEFF_NORM), pero se obtenían peores detecciones.
<br>
Los resultados se muestran en la carpeta output.

In [65]:
import cv2
import numpy as np
import os

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def remove_noise(image):
    denoised_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    return denoised_image

def calculate_self_similarity(template, edge_thresh1, edge_thresh2):
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    gray_template = remove_noise(gray_template)
    edges_template = cv2.Canny(gray_template, edge_thresh1, edge_thresh2)
    result = cv2.matchTemplate(edges_template, edges_template, cv2.TM_CCORR)
    _, max_val, _, _ = cv2.minMaxLoc(result)
    return max_val

def multi_scale_template_matching(image, template, scale_range, angle_range, edge_thresh1, edge_thresh2, n_best_matches=1, self_similarity=None):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    gray_image = remove_noise(gray_image)
    gray_template = remove_noise(gray_template)

    image_height, image_width = gray_image.shape
    template_height, template_width = gray_template.shape
    scale_width = image_width / template_width
    scale_height = image_height / template_height
    scale_factor = min(scale_width, scale_height)
    gray_template = cv2.resize(gray_template, (int(template_width * scale_factor), int(template_height * scale_factor)))

    gray_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges_image = cv2.Canny(gray_image, edge_thresh1, edge_thresh2)    

    best_matches = []

    for scale in scale_range:
        resized_template = cv2.resize(gray_template, (int(gray_template.shape[1] * scale), int(gray_template.shape[0] * scale)))

        if resized_template.shape[0] > gray_image.shape[0] or resized_template.shape[1] > gray_image.shape[1]:
            continue
        
        for angle in angle_range:
            rotated_template = rotate_image(resized_template, angle)
            rotated_template = cv2.GaussianBlur(rotated_template, (5, 5), 0)
            edges_template = cv2.Canny(rotated_template, edge_thresh1, edge_thresh2)
            result = cv2.matchTemplate(edges_image, edges_template, cv2.TM_CCOEFF)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            similarity_score = (max_val / self_similarity) * 100

            if len(best_matches) < n_best_matches:
                best_matches.append((max_val, max_loc, scale, angle, rotated_template, similarity_score))
                best_matches.sort(reverse=True, key=lambda x: x[0])
            elif max_val > best_matches[-1][0]:
                best_matches[-1] = (max_val, max_loc, scale, angle, rotated_template, similarity_score)
                best_matches.sort(reverse=True, key=lambda x: x[0])

    if not best_matches:
        print("No se encontraron buenos matches.")
        return []

    return best_matches

# Directorio que contiene las imágenes
image_directory = 'images'
template = cv2.imread('template/pattern.png', cv2.IMREAD_COLOR)
scale_range = np.linspace(0.02, 1, 100)
angle_range = range(-2, 3)  # De -2 a 2 grados
edge_thresh1 = 100
edge_thresh2 = 200
n_best_matches = 1  # Número de mejores matches que queremos graficar
match_score = 5.0 # Umbral de detección

output_directory = os.path.join(image_directory, 'output')
os.makedirs(output_directory, exist_ok=True)

# Calcular la similitud del template consigo mismo
self_similarity = calculate_self_similarity(template, edge_thresh1, edge_thresh2)

for filename in os.listdir(image_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_directory, filename)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        best_matches = multi_scale_template_matching(image, template, scale_range, angle_range, edge_thresh1, edge_thresh2, n_best_matches, self_similarity)
        
        if best_matches:
            for i, (val, loc, scale, angle, match_template, score) in enumerate(best_matches):
                if score > match_score:
                    top_left = loc
                    template_height, template_width = match_template.shape
                    bottom_right = (top_left[0] + template_width, top_left[1] + template_height)
                    cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
                    cv2.putText(image, f'Match {i+1}: {score:.2f}%', (top_left[0], top_left[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            output_path = os.path.join(output_directory, 'output_' + filename)
            cv2.imwrite(output_path, image)
            print(f'Output guardado en: {output_path} con los {n_best_matches} mejores matches')
        else:
            print(f'No se encontró un buen match para la imagen: {filename}')


Output guardado en: images\output\output_COCA-COLA-LOGO.jpg con los 1 mejores matches
Output guardado en: images\output\output_coca_logo_1.png con los 1 mejores matches
Output guardado en: images\output\output_coca_logo_2.png con los 1 mejores matches
Output guardado en: images\output\output_coca_multi.png con los 1 mejores matches
Output guardado en: images\output\output_coca_retro_1.png con los 1 mejores matches
Output guardado en: images\output\output_coca_retro_2.png con los 1 mejores matches
Output guardado en: images\output\output_logo_1.png con los 1 mejores matches


2. El siguiente código permite detectar las múltiples ocurrencias del logo. Se utiliza otro método de detección, lo que permite individualizar las ocurrencias cuando el umbral está correctamente seteado. 
Se utiliza un algoritmo similar al del punto 1, con detector de bordes y escalado.

In [39]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Leer la imagen principal y el template
img_rgb = cv.imread('images/coca_multi.png')
assert img_rgb is not None, "El archivo no pudo ser leído, verifica con os.path.exists()"
img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
template = cv.imread('template/pattern.png', cv.IMREAD_GRAYSCALE)
assert template is not None, "El archivo no pudo ser leído, verifica con os.path.exists()"

# Definir el rango de escalas a probar
scales = np.linspace(0.2, 1.0, 100)

# Inicializar una lista para almacenar los puntos encontrados
all_points = []

def remove_noise(image):
    # Aplicar un filtro bilateral para eliminar el ruido
    denoised_image = cv.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    return denoised_image

template = remove_noise(template)

img_gray = cv.GaussianBlur(img_gray, (5, 5), 0)
#template = cv.GaussianBlur(template, (5, 5), 0)
img_gray = cv.Canny(img_gray, 150, 200)
template = cv.Canny(template, 150, 200)

# Iterar sobre las diferentes escalas
for scale in scales:
    # Redimensionar el template según la escala actual
    width = int(template.shape[1] * scale)
    height = int(template.shape[0] * scale)
    resized_template = cv.resize(template, (width, height))
    
    # Realizar la búsqueda del template en la imagen
    res = cv.matchTemplate(img_gray, resized_template, cv.TM_CCORR_NORMED)
    threshold = 0.245
    loc = np.where(res >= threshold)
    
    # Almacenar los puntos encontrados
    for pt in zip(*loc[::-1]):
        all_points.append((pt[0], pt[1], pt[0] + width, pt[1] + height, res[pt[1], pt[0]]))

# Aplicar Non-Maximum Suppression (NMS)
def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    score = boxes[:,4]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(score)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

boxes = np.array(all_points)
picked_boxes = non_max_suppression(boxes, 0.3)

# Dibujar los rectángulos en la imagen original
for (x1, y1, x2, y2, _) in picked_boxes:
    cv.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 255), 2)

# Guardar la imagen con los resultados
cv.imwrite('Multi_detect.png', img_rgb)


True

3. Método "generalizado" combina los métodos de los puntos 1 y 2. Primero se intenta con el método del punto 2 y, en caso de no detectar nada, pasa a usar el método del punto 1, lo que logra detectar el logo en todas las imágenes provistas.

In [53]:
import cv2 as cv
import numpy as np
import os

def remove_noise(image):
    denoised_image = cv.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    return denoised_image

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
    return result

def calculate_self_similarity(template, edge_thresh1, edge_thresh2):
    gray_template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    gray_template = remove_noise(gray_template)
    edges_template = cv.Canny(gray_template, edge_thresh1, edge_thresh2)
    result = cv.matchTemplate(edges_template, edges_template, cv.TM_CCORR)
    _, max_val, _, _ = cv.minMaxLoc(result)
    return max_val

def multi_scale_template_matching(image, template, scale_range, angle_range, edge_thresh1, edge_thresh2, n_best_matches=1, self_similarity=None):
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    gray_template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

    gray_image = remove_noise(gray_image)
    gray_template = remove_noise(gray_template)

    image_height, image_width = gray_image.shape
    template_height, template_width = gray_template.shape
    scale_width = image_width / template_width
    scale_height = image_height / template_height
    scale_factor = min(scale_width, scale_height)
    gray_template = cv.resize(gray_template, (int(template_width * scale_factor), int(template_height * scale_factor)))

    gray_image = cv.GaussianBlur(gray_image, (5, 5), 0)
    edges_image = cv.Canny(gray_image, edge_thresh1, edge_thresh2)

    best_matches = []

    for scale in scale_range:
        resized_template = cv.resize(gray_template, (int(gray_template.shape[1] * scale), int(gray_template.shape[0] * scale)))

        if resized_template.shape[0] > gray_image.shape[0] or resized_template.shape[1] > gray_image.shape[1]:
            continue

        for angle in angle_range:
            rotated_template = rotate_image(resized_template, angle)
            rotated_template = cv.GaussianBlur(rotated_template, (5, 5), 0)
            edges_template = cv.Canny(rotated_template, edge_thresh1, edge_thresh2)
            result = cv.matchTemplate(edges_image, edges_template, cv.TM_CCOEFF)
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

            similarity_score = (max_val / self_similarity) * 100

            if len(best_matches) < n_best_matches:
                best_matches.append((max_val, max_loc, scale, angle, rotated_template, similarity_score))
                best_matches.sort(reverse=True, key=lambda x: x[0])
            elif max_val > best_matches[-1][0]:
                best_matches[-1] = (max_val, max_loc, scale, angle, rotated_template, similarity_score)
                best_matches.sort(reverse=True, key=lambda x: x[0])

    if not best_matches:
        print("No se encontraron buenos matches.")
        return []

    return best_matches

def process_images(image_directory, template_path):
    scales = np.linspace(0.2, 1.0, 100)
    edge_thresh1 = 100
    edge_thresh2 = 200
    n_best_matches = 1
    match_score = 5.0
    angle_range = range(-2, 3)
    template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
    assert template is not None, "El archivo no pudo ser leído, verifica con os.path.exists()"
    template = remove_noise(template)

    self_similarity = calculate_self_similarity(cv.imread(template_path, cv.IMREAD_COLOR), edge_thresh1, edge_thresh2)
    
    output_directory = os.path.join(image_directory, 'output')
    os.makedirs(output_directory, exist_ok=True)
    
    for filename in os.listdir(image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_directory, filename)
            img_rgb = cv.imread(image_path, cv.IMREAD_COLOR)
            assert img_rgb is not None, "El archivo no pudo ser leído, verifica con os.path.exists()"
            img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
            img_gray = cv.GaussianBlur(img_gray, (5, 5), 0)
            img_gray = cv.Canny(img_gray, 150, 200)
            template_edges = cv.Canny(template, 150, 200)
            
            all_points = []
            
            for scale in scales:
                width = int(template_edges.shape[1] * scale)
                height = int(template_edges.shape[0] * scale)
                
                if width > img_gray.shape[1] or height > img_gray.shape[0]:
                    continue
                
                resized_template = cv.resize(template_edges, (width, height))
                
                res = cv.matchTemplate(img_gray, resized_template, cv.TM_CCORR_NORMED)
                threshold = 0.27
                loc = np.where(res >= threshold)                
                
                for pt in zip(*loc[::-1]):
                    all_points.append((pt[0], pt[1], pt[0] + width, pt[1] + height, res[pt[1], pt[0]]))
            
            if all_points:
                boxes = np.array(all_points)
                picked_boxes = non_max_suppression(boxes, 0.3)
                
                for (x1, y1, x2, y2, score) in picked_boxes:
                    cv.rectangle(img_rgb, (x1, y1), (x2, y2), (255, 0, 255), 2)
                    cv.putText(img_rgb, f'{score:.2f}', (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)
                
                output_path = os.path.join(output_directory, 'output_' + filename)
                cv.imwrite(output_path, img_rgb)
                print(f'Output guardado en: {output_path}')
            else:
                image = cv.imread(image_path, cv.IMREAD_COLOR)
                template_color = cv.imread(template_path, cv.IMREAD_COLOR)
                best_matches = multi_scale_template_matching(image, template_color, np.linspace(0.02, 1, 100), angle_range, edge_thresh1, edge_thresh2, n_best_matches, self_similarity)
                
                if best_matches:
                    for i, (val, loc, scale, angle, match_template, score) in enumerate(best_matches):
                        if score > match_score:
                            top_left = loc
                            template_height, template_width = match_template.shape
                            bottom_right = (top_left[0] + template_width, top_left[1] + template_height)
                            cv.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
                            cv.putText(image, f'Match {i+1}: {score:.2f}%', (top_left[0], top_left[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    
                    output_path = os.path.join(output_directory, 'output_' + filename)
                    cv.imwrite(output_path, image)
                    print(f'Output guardado en: {output_path} con los {n_best_matches} mejores matches')
                else:
                    print(f'No se encontró un buen match para la imagen: {filename}')

def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    score = boxes[:, 4]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(score)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

# Directorio que contiene las imágenes y ruta del template
image_directory = 'images'
template_path = 'template/pattern.png'

process_images(image_directory, template_path)


Output guardado en: images\output\output_COCA-COLA-LOGO.jpg con los 1 mejores matches
Output guardado en: images\output\output_coca_logo_1.png
Output guardado en: images\output\output_coca_logo_2.png con los 1 mejores matches
Output guardado en: images\output\output_coca_multi.png
Output guardado en: images\output\output_coca_retro_1.png con los 1 mejores matches
Output guardado en: images\output\output_coca_retro_2.png
Output guardado en: images\output\output_logo_1.png con los 1 mejores matches
